In [70]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [76]:
df_in = pd.read_csv('20K_Lending_Club_Loans_base.csv', parse_dates=['loan_app_date'])

In [77]:
df_in['year'] = df_in['loan_app_date'].map(lambda x: x.year)
df_in['month'] = df_in['loan_app_date'].map(lambda x: x.month)
df_in.head()

,loan_id,loan_app_date,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,...,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,is_bad,year,month
0,1,2016-01-10,4000,4000,60 months,7.29%,79.76,A,A4,Time Warner Cable,...,0.0,12087,12.1,44.0,f,NaN,1,0,2016,1
1,2,2016-01-11,16000,16000,60 months,18.25%,408.48,F,F1,Ottawa University,...,0.0,10114,64.0,5.0,f,NaN,1,0,2016,1
2,3,2016-01-12,8700,8700,36 months,7.88%,272.15,A,A5,Kennedy Wilson,...,0.0,81,0.6,8.0,f,NaN,1,0,2016,1
3,4,2016-01-13,18000,18000,60 months,11.49%,395.78,B,B4,TOWN OF PLATTEKILL,...,0.0,10030,37.1,23.0,f,NaN,1,0,2016,1
4,5,2016-01-14,16000,16000,36 months,11.83%,530.15,B,B3,Belmont Correctional,...,0.0,10740,40.4,21.0,f,NaN,1,0,2016,1


In [86]:
lc_cols = ['loan_id', 'loan_app_date', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 
    'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 
    'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'initial_list_status', 'policy_code', 
    'is_bad', 'year', 'month']

credit_cols = ['loan_id', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 
    'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 
    'total_acc', 'mths_since_last_major_derog']

df_lc = df_in[lc_cols]
df_credit = df_in[credit_cols]

In [87]:
df_credit.to_csv('20k_loans_credit_data.csv', index=False)

In [88]:
table1 = pa.Table.from_pandas(df_lc[0:10000], preserve_index=False)
table2 = pa.Table.from_pandas(df_lc[10000:20001], preserve_index=False)

In [91]:
!rm -rf loan_history
pq.write_to_dataset(table1, root_path='loan_history'
    , partition_cols=['year', 'month']
)
pq.write_to_dataset(table2, root_path='loan_history'
    , partition_cols=['year', 'month']
)